In [1]:
from bs4 import BeautifulSoup as bs
import requests
import json

In [2]:
JS_PAPERS_URL = 'https://www.josephsmithpapers.org/the-papers/documents'

In [3]:
parent_html = requests.get(JS_PAPERS_URL).text

soup = bs(parent_html, "html.parser")

soup = soup.find_all("a", {"class": "document-browser__link--menu"}, href=True)

document_category_urls = {}
for a in soup:
    if "volume" not in a["href"]:
        document_category_urls[a.text] = JS_PAPERS_URL + a["href"]

In [4]:
# Headers
headers = {}
# headers["Accept"] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"
headers["Cookie"] = "lds-youtube=true; TAsessionID=5533769a-1c99-4fd2-bcc6-ada392e26896|NEW; notice_behavior=implied|us; s_ips=948; AMCVS_66C5485451E56AAE0A490D45%40AdobeOrg=1; AMCV_66C5485451E56AAE0A490D45%40AdobeOrg=1176715910%7CMCIDTS%7C19456%7CMCMID%7C89507000606349031431620987572725332513%7CMCAAMLH-1681570594%7C9%7CMCAAMB-1681570594%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1680972994s%7CNONE%7CvVersion%7C5.4.0; s_cc=true; analytics_video_metadata_load=false; gpv_Page=documents; gpv_cURL=www.josephsmithpapers.org%2Fthe-papers%2Fdocuments; s_tp=948; s_ppv=documents%2C100%2C100%2C100%2C948%2C1%2C1; adcloud={%22_les_v%22:%22y%2Cjosephsmithpapers.org%2C1680967809%22}; s_plt=0.52; s_pltp=documents; s_sq=ldschurchofjesuschristprod%3D%2526c.%2526a.%2526activitymap.%2526page%253Ddocuments%2526link%253DDocuments%25252C%2525201827%2525E2%252580%2525931829%252520%25252835%252529%2526region%253DBODY%2526pageIDType%253D1%2526.activitymap%2526.a%2526.c%2526pid%253Ddocuments%2526pidt%253D1%2526oid%253Dhttps%25253A%25252F%25252Fwww.josephsmithpapers.org%25252Fthe-papers%25252Fdocuments%25252Fpre1830%2526ot%253DA"

In [12]:
for category, url in document_category_urls.items():
    print(category, url)
    
    document_category_html = requests.get(url, headers=headers).text
    # print(document_category_html)
    if "Cannot read property &#x27;uri&#x27; of undefined" in document_category_html:
        print("Error")
    break
    soup = bs(document_category_html, "html.parser")

    soup = soup.find_all("a", {"class": "document-browser__link"}, href=True)
    print(len(soup))

    document_urls = {}
    for a in soup:
        if "volume" not in a["href"]:
            document_urls[a.text] = JS_PAPERS_URL + a["href"]

    for document, url in document_urls.items():
        print(document, url)
        # document_html = requests.get(url, headers=headers).text
        break
    break



	Documents, 1827–1829  (35) 
 https://www.josephsmithpapers.org/the-papers/documents/the-papers/documents/pre1830
<!DOCTYPE html>
<html lang=>
    <head>
        <!-- Analytics -->
        <script>

            if (false) {
                //prehiding snippet for Adobe Target with asynchronous Launch deployment
                (function(g, b, d, f) {
                    (function (a, c, d) {
                        if (a) {
                            var e = b.createElement('style');
                            e.id = c;
                            e.innerHTML = d;
                            a.appendChild(e);
                        }
                    })(b.getElementsByTagName('head')[0], 'at-body-style', d);

                    setTimeout(function () {
                        var a = b.getElementsByTagName('head')[0];
                        if (a) {
                            var c = b.getElementById('at-body-style');
                            c && a.removeChild(c); }
    

In [9]:
print(document_html)

NameError: name 'document_html' is not defined

In [ ]:


def get_category_and_sub_category_hubs(parent_url) -> dict:
    parent_html = requests.get(parent_url).text

    soup = bs(parent_html, "html.parser")
    soup = soup.find("div", {"class": "main-container container"})
    soup = soup.find("section", {"id": "block-system-main"})
    soup = soup.find("ul", {"class": "menu nav"})

    # Get all of the main categories of documents
    urls_to_visit = {}
    categories = soup.findChildren("li", recursive=False)
    for category in categories:
        category_text = category.find("a").text
        category_name = category_text.split(" (")[0]
        num_in_category = int(category_text.split(" (")[1].replace(")", ""))

        category_href = parent_url.replace("/documents", "") + category.find("a")["href"]
        urls_to_visit[(category_name, None)] = category_href

        # Get all of the subcategories of documents
        sub_category_list = category.find("ul", {"class": "dropdown-menu"})
        if sub_category_list is not None:
            sub_categories = sub_category_list.findChildren("li", recursive=False)
            for sub_category in sub_categories:
                sub_category_text = sub_category.find("a").text
                sub_category_name = sub_category_text.split(" (")[0]
                try:
                    num_in_sub_category = int(sub_category_text.split(" (")[1].replace(")", ""))
                except ValueError:
                    num_in_sub_category = int(sub_category_text.split(" (Radio and Webcast) (")[1].replace(")", ""))

                sub_category_href = parent_url.replace("/documents", "") + sub_category.find("a")["href"]
                urls_to_visit[(category_name, sub_category_name)] = sub_category_href

    return urls_to_visit